In [1]:
#Import statements and sqlContext gets made here

from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField,StringType,MapType,IntegerType
import numpy as np
import re
import sys
from datetime import datetime
sqlContext = SQLContext(sc)


In [2]:
#Paths

cleanCsvsPath = "/home/svanhmic/workspace/Python/Erhvervs/data/regnskabsdata/sparkdata/csv"

In [3]:
def createDfStruct(inputList):
    '''
        Transforms a list of columns to a struct to be parsed into a spark dataframe
        
        Input:
            -inputList: Input list with columns
        Output:
            -outputStruct: StructType() element to be used in a dataframe
    '''
    
    return StructType([StructField(name=n,dataType=StringType(),nullable=True) for n in inputList])
    


In [19]:
def pivotAccountData(df):
    '''
    @summary: Refactoring. Takes all uniqe columns and creates a giant pivot table
    
    Input: 
        df - Spark Data Frame that contains all accounts.
    Output:  
        transformedDf - Spark Data Frame that includes all unique columns meaning one row contains an entire account.
    '''
    removeBlanksDf = (df.filter(df["id"] != np.nan)
                      .select('*',F.regexp_replace(df["value"],r'\s\s+',"").alias("newValue"))
                      .drop("value"))
   
    pivotDataDf = (removeBlanksDf
                   .dropDuplicates()
                   .groupby("id","name","newValue","startDate","endDate","unit")
                   .count()
                   .groupby("id")
                   .pivot("name")
                   .agg(F.max(F.struct("count","newValue"))))
    
    pivotCols = pivotDataDf.columns
    
    pivotValuesDf = (pivotDataDf
                     .rdd
                     .map(lambda x: [None if x[pivotCols[i]] is None else x[pivotCols[i]] if x[pivotCols[i]][1] is None else x[pivotCols[i]][1] for i in range(len(pivotCols))]))
    
    #print(pivotValuesDf.take(2))
    #pivotDataDf.show(50)
    return sqlContext.createDataFrame(data=pivotValuesDf,schema=createDfStruct(pivotCols))
    

In [23]:
def writePivotTable(df,ret=False):
    '''
        This method writes the pivoted data file into a csv file, such that it is not needed to call every time a run
        is made.
        
        Input:
            df - Data frame with columns as values
            
        Output:
            output - csv-file that is saved.
    '''
    ndf = pivotAccountData(df)
    #ndf.show()
    ndf.write.csv(cleanCsvsPath+"/pivotRowDataValues.csv",sep=";",header=True,mode="overwrite")
    #print(ndf.count())
    print("Done")
    if ret is True:
        return ndf

In [24]:
def main():
    '''
    everthing gets executes here
    '''
    
    #Import the csv files 
    df = sqlContext.read.csv(cleanCsvsPath+"/regnskabsdata.csv", sep=";", header=True,encoding="utf-8")
    
    #Pivot the data such that each account becomes one row. All variables are recorded so each column represents a unique variable
    writePivotTable(df)
    
    #Read the pivoed data in again
  
    #typedDf = sqlContext.createDataFrame(newTypeRdd,createDfStruct(ndf.columns))
    #reducedTypesDf = typedDf.distinct()
    #reducedTypesDf.show()
    #print(newTypeRdd.take(2))
    #typedDf.printSchema()
    

In [25]:
if __name__ == '__main__':
    main()
        
    pass

+---+------------------------------------------------+----------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------+---------------------------------------------------------------------+-------------------------------------------------+------------------------------------+------------------------+----------------+--------------------------+---------------+------------------+-------------------------+-----------------------------------------------------------------+---------------------------+----------------------------------------------------------------+----------------------+----------------------------------------------------------+------------------------------------+-------------------------------------------------------------------------+-----------------------+-----------------------------------------+----------------------------+----------------------------------+-

Py4JJavaError: An error occurred while calling o443.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand$$anonfun$run$1.apply$mcV$sp(InsertIntoHadoopFsRelationCommand.scala:149)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand$$anonfun$run$1.apply(InsertIntoHadoopFsRelationCommand.scala:115)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand$$anonfun$run$1.apply(InsertIntoHadoopFsRelationCommand.scala:115)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:115)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apache.sp

In [ ]:
#arr = [["0","A","væk med dig",None],["1","B","hej med dig","False"]]
#testFrameDF = sqlContext.createDataFrame(data=arr)
#testFrameDF.show()

#testFrameDF
#testFrameRdd = testFrameDF.rdd.map(lambda x: testColumnType(*x))
#print(testFrameRdd.collect())

In [ ]:
#rowWithTypeDf = sqlContext.read.csv(cleanCsvsPath+"/columtypedata.csv",sep=";",header=True,encoding="utf-8")
#df = sqlContext.read.csv(cleanCsvsPath+"/regnskabsdata.csv", sep=";", header=True,encoding="utf-8")



In [ ]:
arr = [[x,1] for x in range(0,10)]
newarr = [0 if x[0] == 3 else 2*x[0] if x[0] is 2 else 7  for x in arr]
print(arr)
print(newarr)